In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install tld

In [4]:
import pandas as pd
import numpy as np
import re
from tld import get_fld

In [5]:
path_to_out_2015 = '/content/drive/MyDrive/quotes-2015-filtered_new.json.bz2'
path_to_out_2016 = '/content/drive/MyDrive/quotes-2016-filtered_new.json.bz2'
path_to_out_2017 = '/content/drive/MyDrive/quotes-2017-filtered.json.bz2'
path_to_out_2018 = '/content/drive/MyDrive/quotes-2018-filtered.json.bz2'
path_to_out_2019 = '/content/drive/MyDrive/quotes-2019-filtered.json.bz2'
path_to_out_2020 = '/content/drive/MyDrive/quotes-2020-filtered_new.json.bz2'
data_2015 = pd.read_json(path_to_out_2015, lines=True)
data_2016 = pd.read_json(path_to_out_2016, lines=True)
data_2017 = pd.read_json(path_to_out_2017, lines=True)
data_2018 = pd.read_json(path_to_out_2018, lines=True)
data_2019 = pd.read_json(path_to_out_2019, lines=True)
data_2020 = pd.read_json(path_to_out_2020, lines=True)

In [6]:
data = [data_2015, data_2016, data_2017, data_2018, data_2019, data_2020]
data_all = pd.concat(data)
data_all

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2015-01-14-004988,another layer of burdensome requirements [ tha...,None,[],2015-01-14 19:08:33,1,"[[None, 0.8135], [Mark Brownstein, 0.1865]]",[http://www.motherjones.com/blue-marble/2015/0...,E
1,2015-09-09-008765,As siltation is a natural process in the Meghn...,Anisul Islam Mahmud,[Q23762578],2015-09-09 07:40:37,11,"[[Anisul Islam Mahmud, 0.942], [None, 0.058]]","[http://trust.org/item/20150909073825-39czr, h...",E
2,2015-11-27-010826,"But taking money out of their pockets, I've al...",None,[],2015-11-27 01:45:07,1,"[[None, 0.8499], [phil jackson, 0.1501]]",[http://metrotextual.wordpress.com/2015/11/26/...,E
3,2015-02-09-029204,I want him to be Prime Minister because he sup...,None,[],2015-02-09 04:28:41,1,"[[None, 0.8177], [Malcolm Turnbull, 0.1823]]",[http://sbs.com.au/comedy/article/2015/02/09/w...,E
4,2015-08-20-016545,caused entirely or mostly by human activities.,None,[],2015-08-20 19:35:23,2,"[[None, 0.7455], [Pope Francis, 0.2545]]",[http://www.bloomberg.com/news/articles/2015-0...,E
...,...,...,...,...,...,...,...,...,...
41869,2020-02-15-048259,this balanced red blends weight and power with...,None,[],2020-02-15 15:00:00,1,"[[None, 0.8541], [James Suckling, 0.1459]]",[http://sootoday.com/columns/wine-all-the-time...,E
41870,2020-01-21-085595,This concept of over-regulation is an interest...,Tim Ryan,"[Q10381179, Q3528685, Q513960, Q7804212, Q7804...",2020-01-21 00:00:00,3,"[[Tim Ryan, 0.7641], [None, 0.2359]]",[http://nordic.businessinsider.com/pwc-ceo-sur...,E
41871,2020-01-26-053553,"Well, Joe, you're wrong.",Bernie Sanders,[Q359442],2020-01-26 17:01:39,10,"[[Bernie Sanders, 0.8208], [None, 0.1573], [Jo...",[https://www.nytimes.com/live/2020/iowa-democr...,E
41872,2020-02-23-041538,We've been writing about climate change being ...,Lesley Hughes,[Q53473786],2020-02-23 22:38:42,2,"[[Lesley Hughes, 0.9023], [None, 0.0978]]",[https://www.nytimes.com/2020/02/23/world/aust...,E


In [7]:
data_all['domain_name'] = data_all.urls.map(lambda urls: [get_fld(el) for el in urls])

In [8]:
#nombre de citation pour chaque site internet
listfinal=[]

for el in data_all['domain_name']:
  listfinal.extend(el)
dataf1=pd.DataFrame(pd.Series(listfinal).value_counts())
print(dataf1.loc['insideclimatenews.org'])
print(dataf1.loc['breitbart.com'])

0    21202
Name: insideclimatenews.org, dtype: int64
0    40102
Name: breitbart.com, dtype: int64


In [8]:
#Nonbre de citqtions uniaues pqr journql
listfinal=[]

for el in data_all['domain_name']:
  listfinal.extend(set(el))
dataf=pd.DataFrame(pd.Series(listfinal).value_counts())
print(dataf.loc['insideclimatenews.org'])
print(dataf.loc['breitbart.com'])


0    19973
Name: insideclimatenews.org, dtype: int64
0    23185
Name: breitbart.com, dtype: int64


In [9]:
#nombre de citation utilisé que pour 1 journal
listfinalone=[]
for el in data_all['domain_name']:
  if np.size(el)<=1:
    listfinalone.extend(el)
pd.Series(listfinalone).value_counts().head(20)

dailyclimate.org              7375
carbonbrief.org               6375
insideclimatenews.org         5172
breitbart.com                 3468
theguardian.com               3164
climatechangenews.com         3006
thehill.com                   2929
washingtonexaminer.com        2827
commondreams.org              2807
wordpress.com                 2495
nytimes.com                   2325
cbc.ca                        2302
dailycaller.com               2248
nationalobserver.com          2223
yaleclimateconnections.org    2165
msn.com                       2138
politico.com                  2070
afr.com                       2054
thinkprogress.org             2008
independent.co.uk             1776
dtype: int64

In [10]:
#nombre d'utilisation
listfinaltwo=[]
listfinalone=[]
for el in data_all['domain_name']:
  if el.count('breitbart.com')>=1:
    listfinalone.extend(set(el))
  if el.count('insideclimatenews.org')>=1:
    listfinaltwo.extend(set(el))
breitbart=pd.DataFrame(pd.Series(listfinalone).value_counts())
carbonbrief=pd.DataFrame(pd.Series(listfinaltwo).value_counts())
print(breitbart.head(20))
print(carbonbrief.head(20))

                          0
breitbart.com         23185
yahoo.com              5180
msn.com                4691
news12.com             4370
kdhnews.com            4183
seattletimes.com       3817
sfgate.com             3810
startribune.com        3623
washingtontimes.com    3609
wftv.com               3524
wtop.com               3338
mysanantonio.com       3199
wsbradio.com           3153
wboc.com               3130
ctpost.com             3122
wthr.com               3067
wsoctv.com             3012
stamfordadvocate.com   2967
newstimes.com          2946
phys.org               2914
                                 0
insideclimatenews.org        19973
environmentalhealthnews.org   2675
dailyclimate.org              2644
nytimes.com                   1864
msn.com                       1807
yahoo.com                     1230
thehill.com                   1142
startribune.com                914
seattletimes.com               820
breitbart.com                  805
sfgate.com                 

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib as mpl

breightbart_score = breitbart.loc['breitbart.com']
carbonbrief_score = carbonbrief.loc['breitbart.com']

Dict = {"breitbart.com": 0, "insideclimatenews.org": 1}
nb_top = 70

cmap = plt.cm.Accent
norm = mpl.colors.Normalize(vmin=0, vmax=2)
m = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
G = nx.Graph()
G.add_nodes_from(Dict.keys())

for index, row in breitbart.head(nb_top).iterrows():
  if index != 'breitbart.com':
    Dict[index] = 2
    G.add_weighted_edges_from([('breitbart.com', index, breightbart_score/row)])

for index, row in carbonbrief.head(nb_top).iterrows():
  if index != 'insideclimatenews.org':
    G.add_weighted_edges_from([('insideclimatenews.org', index, carbonbrief_score/row)])
    if index != 'breitbart.com':
      Dict[index] = 2

nodelist,node_color = zip(*Dict.items())
nx.draw_networkx(G, with_labels=False, nodelist=nodelist, node_size=50, node_color=node_color,vmin=0.0,vmax=2.0, cmap=cmap)
plt.show()

In [17]:
import plotly.graph_objects as go
edge_x = []
edge_y = []

pos_ = nx.spring_layout(G)
for edge in G.edges():
    x0, y0 = pos_[edge[0]]
    x1, y1 = pos_[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = pos_[node]
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append(list(G.nodes)[node])#'# of connections: '+str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='Breitbart & CarbonBrief clusters',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.write_html('/content/drive/MyDrive/cluster_graph.html')
fig.show()

In [12]:
#nombre de citation reutiliser uniquement pour le même journal
listfinalb=[]
listfinalc=[]
listfinaln=[]
for el in data_all['domain_name']:
  if np.size(el)>=2:
    if el.count('breitbart.com')==np.size(el):
      listfinalb.extend(set(el))
    if el.count('insideclimatenews.org')==np.size(el):
      listfinalc.extend(set(el))
    if el.count('nytimes.com')==np.size(el):
      listfinaln.extend(set(el))
print(pd.Series(listfinalb).value_counts())
print(pd.Series(listfinalc).value_counts())
print(pd.Series(listfinaln).value_counts())

breitbart.com    440
dtype: int64
insideclimatenews.org    559
dtype: int64
nytimes.com    864
dtype: int64


## Loading wikidata

In [9]:
# Loading wikidata labels
wikidata_file='/content/drive/MyDrive/Project datasets/wikidata_labels_descriptions_quotebank.csv.bz2'
# wikidata_file='/content/drive/MyDrive/Project datasets/wikidata_labels_descriptions.csv.bz2'
wiki_df = pd.read_csv(wikidata_file, compression='bz2', index_col='QID')
wiki_df

,Label,Description
QID,,
Q31,Belgium,country in western Europe
Q45,Portugal,country in southwestern Europe
Q75,Internet,global system of connected computer networks
Q148,People's Republic of China,sovereign state in East Asia
Q155,Brazil,country in South America
...,...,...
Q106302506,didgeridooist,musician who plays the didgeridoo
Q106341153,biochemistry teacher,teacher of biochemistry at any level
Q106368830,2018 Wigan Metropolitan Borough Council electi...,NaN


In [10]:
# Process parties of speakers in the parquet file
import pandas as pd
import numpy as np
wikidata_parquet_file='/content/drive/MyDrive/Project datasets/speaker_attributes.parquet'
parquet_df = pd.read_parquet(wikidata_parquet_file)
parties_df = parquet_df[~(parquet_df.party.isna())]['party']
res = parties_df.map(lambda parties: list(map(lambda party: wiki_df['Label'].get(party), parties)))
res.explode().value_counts().head(25)

# This brieve analysis just give an idea of the parties of the different speakers
# We take into account all parties of the speakers and not only their last one,
# this make sense as they might have been in a different party when they express their opinion

Democratic Party                            28041
Communist Party of the Soviet Union         27920
Republican Party                            26954
Communist Party of China                    14583
Social Democratic Party of Germany          11490
National Socialist German Workers' Party    11033
Christian Democratic Union                   8109
independent politician                       6002
Conservative Party                           5368
Labour Party                                 5242
Liberal Party                                5138
Social Democratic Party                      4586
Indian National Congress                     4347
Polish United Workers' Party                 3659
National Liberal Party                       3642
Centre Party                                 3543
Socialist Unity Party of Germany             3529
Socialist Party                              2889
Liberal Party of Canada                      2809
Communist Party of Germany                   2680


In [11]:
authors = parquet_df.set_index('id')
authors_parties = authors['party']
authors_parties = authors_parties[~(authors_parties.isna())]
authors_parties

id
Q23            [Q327591]
Q207            [Q29468]
Q368           [Q327591]
Q815          [Q1759368]
Q873            [Q29552]
                 ...    
Q106371401     [Q234277]
Q106377030      [Q79854]
Q106377068     [Q234277]
Q106378203       [Q9630]
Q106391184      [Q49768]
Name: party, Length: 399176, dtype: object

In [12]:
breitbart_df = data_all[data_all.domain_name.map(lambda domaines: domaines.count('breitbart.com') > 0)]
carbonbrief_df = data_all[data_all.domain_name.map(lambda domaines: domaines.count('insideclimatenews.org') > 0)]
print(len(breitbart_df))
print(len(carbonbrief_df))

23185
19973


In [13]:
def repeat_qid(qids, n):
  if len(qids) > 0 :
    return [qids[0] for _ in range(n)]
  else:
    return [np.nan for _ in range(n)]

breitbart_speakers_df = breitbart_df.apply(lambda s: repeat_qid(s['qids'], len(list(filter(lambda d: d=='breitbart.com', s['domain_name'])))), axis=1).explode()#.drop_duplicates()
no_speakers_b = breitbart_speakers_df[(breitbart_speakers_df.isna())]
breitbart_speakers_df = breitbart_speakers_df[~(breitbart_speakers_df.isna())]
breitbart_speakers_parties_df = breitbart_speakers_df.map(lambda s:authors_parties.get(s))
breitbart_speakers_parties_df
res_b = breitbart_speakers_parties_df.map(lambda parties: None if parties is None else list(map(lambda party: wiki_df['Label'].get(party), parties)))
print("No speakers :" +str(len(no_speakers_b)))
print("Others :" +str(len(res_b.explode())))
res_b.explode().value_counts().head(10)

No speakers :12463
Others :49125


Republican Party                 15258
Democratic Party                 10214
Independence Party of America     4142
independent politician            1468
Socialist Party                   1036
Christian Democratic Union         468
La République En Marche            466
Democratic Awakening               360
Liberty Union Party                330
Conservative Party                 216
dtype: int64

In [14]:
def repeat_qid(qids, n):
  if len(qids) > 0 :
    return [qids[0] for _ in range(n)]
  else:
    return [np.nan for _ in range(n)]

carbonbrief_speakers_df = carbonbrief_df.apply(lambda s: repeat_qid(s['qids'], len(list(filter(lambda d: d=='insideclimatenews.org', s['domain_name'])))), axis=1).explode()#.drop_duplicates()
no_speakers_c = carbonbrief_speakers_df[(carbonbrief_speakers_df.isna())]

carbonbrief_speakers_df = carbonbrief_speakers_df[~(carbonbrief_speakers_df.isna())]
carbonbrief_speakers_parties_df = carbonbrief_speakers_df.map(lambda s:authors_parties.get(s))
carbonbrief_speakers_parties_df
res_c = carbonbrief_speakers_parties_df.map(lambda parties: None if parties is None else list(map(lambda party: wiki_df['Label'].get(party), parties))[0])
print("No speakers :" +str(len(no_speakers_c)))
print("Others :" +str(len(res_c.explode())))
res_c.explode().value_counts().head(10)

No speakers :8074
Others :13128


Republican Party                           2361
Democratic Party                           2173
Socialist Party                              78
Liberal Party of Canada                      44
La République En Marche                      43
independent politician                       39
Labour Party                                 39
Christian Democratic Union                   33
Australian Labor Party                       28
Minnesota Democratic–Farmer–Labor Party      27
dtype: int64

In [15]:
politics_b=(dataf1.loc['breitbart.com'][0]-len(no_speakers_b))/dataf1.loc['breitbart.com'][0]
typepo_b=res_b.explode().value_counts()
Republicanp_b=typepo_b.loc['Republican Party']/len(res_b.explode())*politics_b
democratep_b=typepo_b.loc['Democratic Party']/len(res_b.explode())*politics_b
Independent_b=typepo_b.loc['Independence Party of America']/len(res_b.explode())*politics_b
Unknowsp_b=len(no_speakers_b)/dataf1.loc['breitbart.com'][0]
other_parties_b=1-(Unknowsp_b+democratep_b+Republicanp_b+Independent_b)

In [16]:
politics_c=len(res_c.explode())/dataf1.loc['insideclimatenews.org'][0]
typepo_c=res_c.explode().value_counts()
Republicanp_c=typepo_c.loc['Republican Party']/len(res_c.explode())*politics_c
democratep_c=typepo_c.loc['Democratic Party']/len(res_c.explode())*politics_c
Unknowsp_c=len(no_speakers_c)/dataf1.loc['insideclimatenews.org'][0]
other_parties_c=1-(Unknowsp_c+democratep_c+Republicanp_c)

In [33]:
import plotly.graph_objects as go
labels = ['Republicains','Democrats','Unknown','Other Parties','Independent']
values = [Republicanp_c*100, democratep_c*100, Unknowsp_c*100,other_parties_c*100]
colors=['red','blue','green','yellow','purple']
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_traces(marker=dict(colors=colors))
fig.write_html('/content/drive/MyDrive/pieinside.html')
fig.show()


In [34]:
values = [Republicanp_b*100, democratep_b*100, Unknowsp_b*100,other_parties_b*100,Independent_b*100]

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_traces(marker=dict(colors=colors))
fig.write_html('/content/drive/MyDrive/pie_breitbart_final.html')
fig.show()